# Introduction to DeepPavlov
In this tutorial we will learn how to install and construct a simple bot based on pattern matching and the library abstactions.

### Libraries

For this task you will need the following libraries:
 - [Tensorflow](https://www.tensorflow.org) — an open-source software library for Machine Intelligence.
 - [Numpy](http://www.numpy.org) — a package for scientific computing.
 - [DeepPavlov](https://github.com/deepmipt/deeppavlov) - open source library for Natural Language Processing
 


# Installation of DeepPavlov library

Currently only [Linux, Windows] platforms and Python 3.6 are supported

- Create a virtual environment with Python 3.6

    `virtualenv env`

- Activate the environment (Linux):

    `source ./env/bin/activate`

- Activate the environment (Windows):

    `call env\Scripts\activate.bat`

- Install the package inside created virtual environment

    `pip install deeppavlov`

# Run the library using Docker

First, install the Docker.

Then go to console and get the container with the following command:

`docker pull deeppavlov/jupyter`

Run the container:

`docker run -p 8888:8888 deeppavlov/jupyter`

Navigate to http://127.0.0.1:8888/ in your browser.

To STOP the container run:

`docker stop`

# HelloBot

In this part we will construct a simple bot that relies on pattern matching to perform a conversation.

In [2]:
from deeppavlov.agents.default_agent.default_agent import DefaultAgent
from deeppavlov.agents.processors.highest_confidence_selector import HighestConfidenceSelector
from deeppavlov.skills.pattern_matching_skill import PatternMatchingSkill

A pattern matching skill is the simplest example of Natural Language Understanding component. It will search defined patterns through the text. Let's define some simple patterns:

In [3]:
skill_hello = PatternMatchingSkill(['Hello world'], patterns=['hi', 'hello', 'good day'])
skill_bye = PatternMatchingSkill(['Goodbye world', 'See you around'], patterns=['bye', 'chao', 'see you'])
skill_fallback = PatternMatchingSkill(['I don\'t understand, sorry', 'I can say "Hello world"'])

If you provide some patterns to the PatternMatchingSkill it will return confidence = 1 when the skill finds the pattern in given text. If no patterns is provided then confidence 0.5 will be returned in any case.

The skills are used in the `Agent` which can be treated as a Dialog Manager. The agent must be provided with skills and the processor of skills. A simple skill processor is the HighestConfidenceSelector which will pick the skill with highest confidence.

In [4]:
HelloBot = DefaultAgent([skill_hello, skill_bye, skill_fallback], skills_processor=HighestConfidenceSelector())

Since all processing in the library is performed on batches, we can pass a batch of requests to the bot. Let's try it out:

In [5]:
HelloBot(['Hello!', 'Boo...', 'chao'])

['Hello world', "I don't understand, sorry", 'Goodbye world']

In [6]:
#Sometimes the Agent choose to respond with another answer even if the question the pattern didn't change
#(randomly choosen ?)
HelloBot(['Hello!', 'Boo...', 'chao.'])

['Hello world', 'I can say "Hello world"', 'Goodbye world']

**Exercise** 
- create a WhatIsYourName skill
- create new agent with this skill
- check that all works fine

In [7]:
#Creation of WhatIsYourName skill
skill_name = PatternMatchingSkill(['My name is Pavlov'], patterns=["What is your name ?", "your name ?",
                                                                   "Do you have a name ?"])
skill_fallback = PatternMatchingSkill(['I don\'t understand, sorry', 'I can say "Hello world"'])



In [8]:
#Creation of the a new agent only for the WhatIsYourName skill
NameBot = DefaultAgent([skill_name,skill_fallback], skills_processor=HighestConfidenceSelector())

In [9]:
NameBot(["What's your name ?"])

['My name is Pavlov']

In [10]:
#One of the limitations of this simple Agent is that he need to find the full pattern
#to be able to do the matching
#in this case the "?" is missing

NameBot(["What's your name "])

["I don't understand, sorry"]

In [11]:
#Whenever he finds the full pattern he answers without taking in account the other things
NameBot([" 1234 What's your name ? 5678aeifsjdkzekjbj "])

['My name is Pavlov']